In [1]:
import logging
from eppy.modeleditor import IDF
import glob
import pandas as pd
import numpy as np
import pyumi as pu

In [2]:
files = glob.glob("data/necb/NECB_2011_Montreal_idf/*.idf")

In [3]:
files

['data/necb/NECB_2011_Montreal_idf\\NECB 2011-FullServiceRestaurant-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf',
 'data/necb/NECB_2011_Montreal_idf\\NECB 2011-HighriseApartment-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf',
 'data/necb/NECB_2011_Montreal_idf\\NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf',
 'data/necb/NECB_2011_Montreal_idf\\NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf']

In [4]:
logging.info("start")

In [5]:
idfs = []
# Windows
iddfile = "C:\openstudio-2.5.2\EnergyPlus\Energy+.idd"
# Mac
# iddfile = "/Applications/OpenStudio-2.5.0/EnergyPlus/Energy+.idd"
epw = 'CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw'
IDF.setiddname(iddfile)
for file in files:
    idfs.append(IDF(file, epw))

In [6]:
def get_values(frame):
    ncols = min(len(frame.fieldvalues),len(frame.fieldnames))
    return pd.DataFrame([frame.fieldvalues[0:ncols]], columns=frame.fieldnames[0:ncols])

In [7]:
keys = [idf.idfobjects['BUILDING'].list2[0][1] for idf in idfs]

In [8]:
# for idf in idfs:
#     for obj in idf.idfobjects:
#         print(obj)

## GasMaterials

In [9]:
GasMaterialss = []
for idf in idfs:
    GasMaterials = idf.idfobjects['WINDOWMATERIAL:GAS']
    GasMaterials = [get_values(frame) for frame in GasMaterials]
    GasMaterials = pd.concat(GasMaterials, ignore_index=True)
    GasMaterialss.append(GasMaterials)
GasMaterials = pd.concat(GasMaterialss, keys=keys, names=['Archetype','$id'])
GasMaterials = GasMaterials.groupby('Name').first()
GasMaterials.reset_index(inplace=True)

In [10]:
GasMaterials.index.rename('$id', inplace=True)

In [11]:
GasMaterials['Cost'] = 0
GasMaterials['EmbodiedCarbon'] = 0
GasMaterials['EmbodiedCarbonStdDev'] = 0
GasMaterials['EmbodiedEnergy'] = 0
GasMaterials['EmbodiedEnergyStdDev'] = 0
GasMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
GasMaterials['SubstitutionTimestep'] = 0
GasMaterials['TransportCarbon'] = 0
GasMaterials['TransportDistance'] = 0
GasMaterials['TransportEnergy'] = 0

In [12]:
def gas_type(row):
    if 'air' in row['Gas_Type'].lower():
        return 0
    elif 'argon' in row['Gas_Type']:
        return 1
    elif 'krypton' in row['Gas_Type'].lower():
        return 2
    elif 'xenon' in row['Gas_Type'].lower():
        return 3
    elif 'sf6' in row['Gas_Type'].lower():
        return 4

In [13]:
# Add GasType Column based on previous function
GasMaterials['GasType'] = GasMaterials.apply(lambda x: gas_type(x), axis=1)

In [14]:
# Remove unnecessary columns
GasMaterials.drop(['key','Gas_Type', 'Thickness'], axis=1, inplace=True)

In [15]:
GasMaterials

,Name,Cost,EmbodiedCarbon,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,GasType
$id,,,,,,,,,,,,
0,AIR 13MM,0,0,0,0,0,NaN,0,0,0,0,0
1,AIR 6MM,0,0,0,0,0,NaN,0,0,0,0,0


In [16]:
# THis is an example and should be deleted
GasMaterialss = []
for idf in idfs:
    GasMaterials = idf.idfobjects['WINDOWMATERIAL:GAS']
    GasMaterials = [get_values(frame) for frame in GasMaterials]
    GasMaterials = pd.concat(GasMaterials, ignore_index=True)
    GasMaterialss.append(GasMaterials)
GasMaterials = pd.concat(GasMaterialss, keys=keys, names=['Archetype','$id'])
GasMaterials = GasMaterials.groupby('Name').first()
GasMaterials.reset_index(inplace=True)

In [17]:
def newrange(previous, following):
    From = previous.iloc[[-1]].index.values + 1
    To = From + len(following)
    return np.arange(From, To)

## GlazingMaterials

#### WindowMaterial

In [18]:
WindowMaterial_Glazings = []
for idf in idfs:
    WindowMaterial_Glazing = idf.idfobjects['WINDOWMATERIAL:GLAZING']
    WindowMaterial_Glazing = [get_values(frame) for frame in WindowMaterial_Glazing]
    WindowMaterial_Glazing = pd.concat(WindowMaterial_Glazing, ignore_index=True, sort=True)
    WindowMaterial_Glazings.append(WindowMaterial_Glazing)
WindowMaterial_Glazing = pd.concat(WindowMaterial_Glazings, keys=keys, names=['Archetype','$id'])
WindowMaterial_Glazing = WindowMaterial_Glazing.groupby('Name').first()
WindowMaterial_Glazing.reset_index(inplace=True)

In [19]:
column_rename = {'Optical_Data_Type':'Optical',
                'Window_Glass_Spectral_Data_Set_Name':'OpticalData',
                'Solar_Transmittance_at_Normal_Incidence':'SolarTransmittance',
                'Front_Side_Solar_Reflectance_at_Normal_Incidence':'SolarReflectanceFront',
                'Back_Side_Solar_Reflectance_at_Normal_Incidence':'SolarReflectanceBack',
                'Infrared_Transmittance_at_Normal_Incidence':'IRTransmittance',
                'Visible_Transmittance_at_Normal_Incidence':'VisibleTransmittance',
                'Front_Side_Visible_Reflectance_at_Normal_Incidence':'VisibleReflectanceFront',
                'Back_Side_Visible_Reflectance_at_Normal_Incidence':'VisibleReflectanceBack',
                'Front_Side_Infrared_Hemispherical_Emissivity':'IREmissivityFront',
                'Back_Side_Infrared_Hemispherical_Emissivity':'IREmissivityBack',
                'Dirt_Correction_Factor_for_Solar_and_Visible_Transmittance':'DirtFactor'}

In [20]:
GlazingMaterials = WindowMaterial_Glazing
GlazingMaterials.rename(columns=column_rename, inplace=True)
GlazingMaterials = GlazingMaterials.drop(['key', 'Solar_Diffusing'], axis=1)
GlazingMaterials['Comment'] = 'default'
GlazingMaterials['Cost'] = 0
GlazingMaterials['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]
GlazingMaterials['Density'] = 2500
GlazingMaterials['EmbodiedCarbon'] = 0
GlazingMaterials['EmbodiedCarbonStdDev'] = 0
GlazingMaterials['EmbodiedEnergy'] = 0
GlazingMaterials['EmbodiedEnergyStdDev'] = 0
GlazingMaterials['Life'] = 1
GlazingMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
GlazingMaterials['SubstitutionTimestep'] = 0
GlazingMaterials['TransportCarbon'] = 0
GlazingMaterials['TransportDistance'] = 0
GlazingMaterials['TransportEnergy'] = 0
GlazingMaterials['Type'] = 'Uncoated' # TODO Further investigation necessary

In [21]:
GlazingMaterials.index = newrange(GasMaterials, GlazingMaterials)
GlazingMaterials.index.rename('$id', inplace=True)
GlazingMaterials

,Name,IREmissivityBack,SolarReflectanceBack,VisibleReflectanceBack,Conductivity,DirtFactor,IREmissivityFront,SolarReflectanceFront,VisibleReflectanceFront,IRTransmittance,...,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,Life,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,Type
$id,,,,,,,,,,,,,,,,,,,,,
2,CLEAR 6MM,0.840,0.071,0.080,0.9000,1.0,0.840,0.0710,0.0800,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
3,Clear 3mm,0.840,0.075,0.081,0.9000,1.0,0.840,0.0750,0.0810,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
4,LoE TINT 6MM,0.100,0.200,0.054,0.9000,1.0,0.840,0.0930,0.0350,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
5,REF D CLEAR 6MM,0.820,0.379,0.505,0.9000,1.0,0.840,0.3080,0.4530,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
6,REF D TINT 6MM,0.820,0.360,0.450,0.9000,1.0,0.840,0.1400,0.1800,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
7,Theoretical Glass [167],0.985,0.000,0.000,2.1073,1.0,0.985,0.7126,0.6988,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
8,Theoretical Glass [197],0.900,0.000,0.000,0.0415,1.0,0.900,0.7151,0.6988,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
9,Theoretical Glass [202],0.900,0.000,0.000,0.0192,1.0,0.900,0.7175,0.6308,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated
10,Theoretical Glass [207],0.900,0.000,0.000,0.0133,1.0,0.900,0.6189,0.5100,0.0,...,0,0,0,1,NaN,0,0,0,0,Uncoated


#### WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM

In [22]:
WindowMaterialsSimpleGlazes = []
for idf in idfs:
    WindowMaterialsSimpleGlaze = idf.idfobjects['WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM']
    WindowMaterialsSimpleGlaze = [get_values(frame) for frame in WindowMaterialsSimpleGlaze]
    WindowMaterialsSimpleGlaze = pd.concat(WindowMaterialsSimpleGlaze, ignore_index=True, sort=True)
    WindowMaterialsSimpleGlazes.append(WindowMaterialsSimpleGlaze)
WindowMaterialsSimpleGlaze = pd.concat(WindowMaterialsSimpleGlazes, keys=keys, names=['Archetype','$id'])
WindowMaterialsSimpleGlaze = WindowMaterialsSimpleGlaze.groupby('Name').first()
WindowMaterialsSimpleGlaze.reset_index(inplace=True)

In [23]:
WindowMaterialsSimpleGlaze.columns

Index(['Name', 'Solar_Heat_Gain_Coefficient', 'UFactor',
       'Visible_Transmittance', 'key'],
      dtype='object')

In [24]:
SimpleGlazing = WindowMaterialsSimpleGlaze.apply(lambda row: pu.simple_glazing(row['Solar_Heat_Gain_Coefficient'], row['UFactor'], row['Visible_Transmittance']), axis=1).apply(pd.Series)
SimpleGlazing.loc[:,'Name'] = WindowMaterialsSimpleGlaze['Name']
SimpleGlazing.loc[:,'DataSource'] = 'EnergyPlus Simple Glazing Calculation'

In [25]:
WindowMaterialsSimpleGlaze.iloc[0]

Name                           Customized Fenestration:: cond=0.220 tvis=0.22...
Solar_Heat_Gain_Coefficient                                                  0.6
UFactor                                                                      2.2
Visible_Transmittance                                                       0.21
key                                           WindowMaterial:SimpleGlazingSystem
Name: 0, dtype: object

In [26]:
SimpleGlazing.Name.values

array(['Customized Fenestration:: cond=0.220 tvis=0.220 tsol=0.232',
       'Customized Fenestration:: cond=0.220 tvis=0.294 tsol=0.332',
       'Customized Fenestration:: cond=0.220 tvis=0.441 tsol=0.279',
       'Customized Fenestration:: cond=0.220 tvis=0.898 tsol=0.837'],
      dtype=object)

In [27]:
GlazingMaterials.append(SimpleGlazing, ignore_index=True, sort=True)

,Comment,Conductivity,Cost,DataSource,Density,DirtFactor,EmbodiedCarbon,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,...,SubstitutionRatePattern,SubstitutionTimestep,Thickness,TransportCarbon,TransportDistance,TransportEnergy,Type,VisibleReflectanceBack,VisibleReflectanceFront,VisibleTransmittance
0,default,0.900000,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.006000,0,0,0,Uncoated,0.080000,0.080000,0.8810
1,default,0.900000,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.003000,0,0,0,Uncoated,0.081000,0.081000,0.8980
2,default,0.900000,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.006000,0,0,0,Uncoated,0.054000,0.035000,0.5000
3,default,0.900000,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.006000,0,0,0,Uncoated,0.505000,0.453000,0.3340
4,default,0.900000,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.006000,0,0,0,Uncoated,0.450000,0.180000,0.2500
5,default,2.107300,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.003000,0,0,0,Uncoated,0.000000,0.698800,0.2512
6,default,0.041500,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.003000,0,0,0,Uncoated,0.000000,0.698800,0.2512
7,default,0.019200,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.003000,0,0,0,Uncoated,0.000000,0.630800,0.3192
8,default,0.013300,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.003000,0,0,0,Uncoated,0.000000,0.510000,0.4400
9,default,0.013300,0,NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ...,2500,1.0,0,0,0,0,...,NaN,0,0.003000,0,0,0,Uncoated,0.000000,0.442100,0.5079


## OpaqueMaterials

In [28]:
# MASS
MaterialsMass_s = []
for idf in idfs:
    MaterialsMass = idf.idfobjects['MATERIAL']
    MaterialsMass = [get_values(frame) for frame in MaterialsMass]
    MaterialsMass = pd.concat(MaterialsMass, ignore_index=True, sort=True)
    MaterialsMass_s.append(MaterialsMass)
MaterialsMass = pd.concat(MaterialsMass_s, keys=keys, names=['Archetype','$id'])
MaterialsMass = MaterialsMass.groupby('Name').first()
MaterialsMass.reset_index(inplace=True)

In [49]:
MaterialsMass

,Name,Conductivity,Density,Roughness,Solar_Absorptance,Specific_Heat,Thermal_Absorptance,Thickness,Visible_Absorptance,key
0,1/2IN Gypsum,0.1600,784.90,Smooth,0.40,830.0,0.90,0.0127,0.40,Material
1,1/2IN Gypsum 1,0.1600,784.90,Smooth,0.40,830.0,0.90,0.0127,0.40,Material
2,100mm Normalweight concrete floor,2.3100,2322.00,MediumSmooth,0.70,832.0,0.90,0.1016,0.70,Material
3,1IN Stucco,0.6918,1858.00,Smooth,0.70,837.0,0.90,0.0253,0.92,Material
4,1IN Stucco 1,0.6918,1858.00,Smooth,0.70,837.0,0.90,0.0253,0.92,Material
5,4 in. Normalweight Concrete Floor,2.3100,2322.00,MediumRough,0.70,832.0,0.90,0.1016,0.70,Material
6,4 in. Normalweight Concrete Floor 1,2.3100,2322.00,MediumRough,0.70,832.0,0.90,0.1016,0.70,Material
7,6 in. Normalweight Concrete Floor,2.3100,2322.00,MediumRough,0.70,832.0,0.90,0.1524,0.70,Material
8,6 in. Normalweight Concrete Floor 1,2.3100,2322.00,MediumRough,0.70,832.0,0.90,0.1524,0.70,Material
9,6 in. Normalweight Concrete Floor 2,2.3100,2322.00,MediumRough,0.70,832.0,0.90,0.1524,0.70,Material


In [29]:
# NoMASS
MaterialNoMass_s = []
for idf in idfs:
    MaterialNoMass = idf.idfobjects['MATERIAL:NOMASS']
    MaterialNoMass = [get_values(frame) for frame in MaterialNoMass]
    MaterialNoMass = pd.concat(MaterialNoMass, ignore_index=True, sort=True)
    MaterialNoMass_s.append(MaterialNoMass)
MaterialNoMass = pd.concat(MaterialNoMass_s, keys=keys, names=['Archetype','$id'])
MaterialNoMass = MaterialNoMass.groupby('Name').first()
MaterialNoMass.reset_index(inplace=True)

In [50]:
MaterialNoMass

,Name,Roughness,Solar_Absorptance,Thermal_Absorptance,Thermal_Resistance,Visible_Absorptance,key
0,CP02 CARPET PAD,VeryRough,0.7,0.9,0.216480,0.8,Material:NoMass
1,CP02 CARPET PAD 1,VeryRough,0.7,0.9,0.216480,0.8,Material:NoMass
2,Nonres_Floor_Insulation,MediumSmooth,0.7,0.9,2.882920,0.7,Material:NoMass
3,Typical Carpet Pad,Smooth,0.7,0.9,0.216480,0.8,Material:NoMass
4,Typical Carpet Pad 1,Smooth,0.7,0.9,0.216480,0.8,Material:NoMass
5,Typical Carpet Pad 2,Smooth,0.7,0.9,1.255030,0.8,Material:NoMass
6,Typical Carpet Pad 3,Smooth,0.7,0.9,3.455153,0.8,Material:NoMass
7,Typical Insulation R-0.58,Smooth,0.7,0.9,0.101875,0.7,Material:NoMass
8,Typical Insulation R-0.58 1,Smooth,0.7,0.9,0.454528,0.7,Material:NoMass
9,Typical Insulation R-1.15,Smooth,0.7,0.9,0.202527,0.7,Material:NoMass


In [51]:
OpaqueMaterials = pd.concat([MaterialsMass,MaterialNoMass], sort=True, ignore_index=True)
OpaqueMaterials.index = newrange(GlazingMaterials, OpaqueMaterials)
OpaqueMaterials.index.name = '$id'

In [52]:
column_rename = {'Solar_Absorptance':'SolarAbsorptance',
                'Specific_Heat':'SpecificHeat', 
                'Thermal_Absorptance':'ThermalEmittance',
                'Thermal_Resistance':'ThermalResistance',
                'Visible_Absorptance':'VisibleAbsorptance'}

In [53]:
OpaqueMaterials.rename(columns=column_rename, inplace=True)

In [54]:
OpaqueMaterials['Comment'] = 'default'
OpaqueMaterials['Cost'] = 0
OpaqueMaterials['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]
OpaqueMaterials['EmbodiedCarbon'] = 0
OpaqueMaterials['EmbodiedCarbonStdDev'] = 0
OpaqueMaterials['EmbodiedEnergy'] = 0
OpaqueMaterials['EmbodiedEnergyStdDev'] = 0
OpaqueMaterials['Life'] = 1
OpaqueMaterials['MoistureDiffusionResistance'] = 50
OpaqueMaterials['PhaseChange'] = False
OpaqueMaterials['PhaseChangeProperties'] = '' # ! Further investigation needed
OpaqueMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
OpaqueMaterials['SubstitutionTimestep'] = 0
OpaqueMaterials['TransportCarbon'] = 0
OpaqueMaterials['TransportDistance'] = 0
OpaqueMaterials['TransportEnergy'] = 0
OpaqueMaterials['Type'] = '' # ! Further investigation necessary
OpaqueMaterials['VariableConductivity'] = False
OpaqueMaterials['VariableConductivityProperties'] = np.NaN # ! Further investigation necessary

In [55]:
OpaqueMaterials = OpaqueMaterials.drop(['key'], axis=1)

In [56]:
OpaqueMaterials

,Conductivity,Density,Name,Roughness,SolarAbsorptance,SpecificHeat,ThermalEmittance,ThermalResistance,Thickness,VisibleAbsorptance,...,PhaseChange,PhaseChangeProperties,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,Type,VariableConductivity,VariableConductivityProperties
$id,,,,,,,,,,,,,,,,,,,,,
13,0.1600,784.90,1/2IN Gypsum,Smooth,0.40,830.0,0.90,NaN,0.0127,0.40,...,False,,NaN,0,0,0,0,,False,NaN
14,0.1600,784.90,1/2IN Gypsum 1,Smooth,0.40,830.0,0.90,NaN,0.0127,0.40,...,False,,NaN,0,0,0,0,,False,NaN
15,2.3100,2322.00,100mm Normalweight concrete floor,MediumSmooth,0.70,832.0,0.90,NaN,0.1016,0.70,...,False,,NaN,0,0,0,0,,False,NaN
16,0.6918,1858.00,1IN Stucco,Smooth,0.70,837.0,0.90,NaN,0.0253,0.92,...,False,,NaN,0,0,0,0,,False,NaN
17,0.6918,1858.00,1IN Stucco 1,Smooth,0.70,837.0,0.90,NaN,0.0253,0.92,...,False,,NaN,0,0,0,0,,False,NaN
18,2.3100,2322.00,4 in. Normalweight Concrete Floor,MediumRough,0.70,832.0,0.90,NaN,0.1016,0.70,...,False,,NaN,0,0,0,0,,False,NaN
19,2.3100,2322.00,4 in. Normalweight Concrete Floor 1,MediumRough,0.70,832.0,0.90,NaN,0.1016,0.70,...,False,,NaN,0,0,0,0,,False,NaN
20,2.3100,2322.00,6 in. Normalweight Concrete Floor,MediumRough,0.70,832.0,0.90,NaN,0.1524,0.70,...,False,,NaN,0,0,0,0,,False,NaN
21,2.3100,2322.00,6 in. Normalweight Concrete Floor 1,MediumRough,0.70,832.0,0.90,NaN,0.1524,0.70,...,False,,NaN,0,0,0,0,,False,NaN


## OpaqueConstructions

In [36]:
# Thermal Resistance : (m2-K)/W
# Conductivity : W/(m-K)

In [64]:
Constructions = []
for idf in idfs:  
    Construction = idf.idfobjects['CONSTRUCTION']
    Construction = [get_values(frame) for frame in Construction]
    Construction = pd.concat(Construction, ignore_index=True, sort=True)
    Constructions.append(Construction)
    Construction.index = newrange(OpaqueMaterials, Construction)
Constructions = pd.concat(Constructions, keys=keys, names=['Archetype','$id'])
Constructions = Constructions.groupby('Name').first()
Constructions.reset_index(inplace=True)

In [65]:
Constructions

,Name,Layer_2,Layer_3,Layer_4,Outside_Layer,key
0,Basement Floor construction,CP02 CARPET PAD,NaN,NaN,M10 200mm concrete block basement wall,Construction
1,Basement Wall construction,NaN,NaN,NaN,M10 200mm concrete block basement wall,Construction
2,Customized Fenestration: cond=0.220 tvis=0.220...,NaN,NaN,NaN,Customized Fenestration:: cond=0.220 tvis=0.22...,Construction
3,Customized Fenestration: cond=0.220 tvis=0.294...,NaN,NaN,NaN,Customized Fenestration:: cond=0.220 tvis=0.29...,Construction
4,Customized Fenestration: cond=0.220 tvis=0.441...,NaN,NaN,NaN,Customized Fenestration:: cond=0.220 tvis=0.44...,Construction
5,Customized Fenestration: cond=0.220 tvis=0.898...,NaN,NaN,NaN,Customized Fenestration:: cond=0.220 tvis=0.89...,Construction
6,Customized opaque construction {058cb932-e200-...,Typical Insulation R-0.58 1,NaN,NaN,F08 Metal surface 1,Construction
7,Customized opaque construction {0978471f-1a2b-...,Typical Insulation R-0.58 1,NaN,NaN,F08 Metal surface 1,Construction
8,Customized opaque construction {1292baf7-9fd7-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,1IN Stucco 1,Construction
9,Customized opaque construction {3a0bc7d3-1520-...,Typical Insulation R-19.63 1,NaN,NaN,Metal Roof Surface 1,Construction


In [66]:
OpaqueConstructions_s = []
for idf in idfs:
    OpaqueConstructions = idf.idfobjects['BUILDINGSURFACE:DETAILED']
    OpaqueConstructions = [get_values(frame) for frame in OpaqueConstructions]
    OpaqueConstructions = pd.concat(OpaqueConstructions, ignore_index=True, sort=True)
    OpaqueConstructions = OpaqueConstructions.merge(Constructions, left_on='Construction_Name', right_on='Name')
    OpaqueConstructions_s.append(OpaqueConstructions)
OpaqueConstructions = pd.concat(OpaqueConstructions_s, keys=keys, names=['Archetype','$id'], sort=True)
OpaqueConstructions = OpaqueConstructions.groupby('Construction_Name').first()
OpaqueConstructions.reset_index(inplace=True)

In [69]:
OpaqueConstructions

,Construction_Name,Layer_2,Layer_3,Layer_4,Name_x,Name_y,Number_of_Vertices,Outside_Boundary_Condition,Outside_Boundary_Condition_Object,Outside_Layer,...,Vertex_8_Ycoordinate,Vertex_8_Zcoordinate,Vertex_9_Xcoordinate,Vertex_9_Ycoordinate,Vertex_9_Zcoordinate,View_Factor_to_Ground,Wind_Exposure,Zone_Name,key_x,key_y
0,Customized opaque construction {1292baf7-9fd7-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,Dining_Wall_East,Customized opaque construction {1292baf7-9fd7-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-Dining Sys-3 Flr-1 Sch-B HPlcmt-south ZN,BuildingSurface:Detailed,Construction
1,Customized opaque construction {3a0bc7d3-1520-...,Typical Insulation R-19.63 1,NaN,NaN,Bath_ZN_1_FLR_1_Ceiling,Customized opaque construction {3a0bc7d3-1520-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-Bath_ZN_1_FLR_1 Sys-3 Flr-1 Sch-D HPlcmt-we...,BuildingSurface:Detailed,Construction
2,Customized opaque construction {47144b0f-1dcc-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,Bath_ZN_1_FLR_1_Wall_5,Customized opaque construction {47144b0f-1dcc-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-Bath_ZN_1_FLR_1 Sys-3 Flr-1 Sch-D HPlcmt-we...,BuildingSurface:Detailed,Construction
3,Customized opaque construction {4b8be161-d6b6-...,Typical Carpet Pad 2,NaN,NaN,g Floor C,Customized opaque construction {4b8be161-d6b6-...,,Ground,,6 in. Normalweight Concrete Floor 1,...,NaN,NaN,NaN,NaN,NaN,,NoWind,Sp-G Corridor Sys-1 Flr-5 Sch-G HPlcmt-east ZN,BuildingSurface:Detailed,Construction
4,Customized opaque construction {70a08f41-941f-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,F2 EWall C,Customized opaque construction {70a08f41-941f-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-F2 Corridor Sys-1 Flr-7 Sch-G HPlcmt-east ZN,BuildingSurface:Detailed,Construction
5,Customized opaque construction {8d7418df-8433-...,Typical Insulation R-19.63 1,NaN,NaN,t Roof C,Customized opaque construction {8d7418df-8433-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-T Corridor Sys-1 Flr-4 Sch-G HPlcmt-east ZN,BuildingSurface:Detailed,Construction
6,Customized opaque construction {a4a6387c-c76b-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,Corridor_Flr_1-1,Customized opaque construction {a4a6387c-c76b-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-Corridor_Flr_1 Sys-6 Flr-1 Sch-B HPlcmt-nor...,BuildingSurface:Detailed,Construction
7,Customized opaque construction {d71d378f-b843-...,Typical Carpet Pad 2,NaN,NaN,Dining_Floor,Customized opaque construction {d71d378f-b843-...,,Ground,,6 in. Normalweight Concrete Floor 1,...,NaN,NaN,NaN,NaN,NaN,,NoWind,Sp-Dining Sys-3 Flr-1 Sch-B HPlcmt-south ZN,BuildingSurface:Detailed,Construction
8,Customized opaque construction {dae5c91d-b614-...,Typical Insulation R-19.63 1,NaN,NaN,east-roof,Customized opaque construction {dae5c91d-b614-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,NaN,NaN,,WindExposed,Sp-attic Sys-0 Flr-2 Sch-- undefined - HPlcmt-...,BuildingSurface:Detailed,Construction
9,Customized opaque construction {dd65f4c9-6f6c-...,Typical Insulation R-19.63 1,NaN,NaN,Corridor_Flr_5-13,Customized opaque construction {dd65f4c9-6f6c-...,,Outdoors,,Metal Roof Surface 1,...,-7.622,4.2672,NaN,NaN,NaN,,WindExposed,Sp-Corridor_Flr_5 Sys-3 Flr-6 Sch-B HPlcmt-nor...,BuildingSurface:Detailed,Construction


In [41]:
def label_surface(row):
    """
    This function adds the umi-Category column
    """
    # Floors
    if row['Surface_Type'] == 'Floor':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 'Interior Floor'
        if row['Outside_Boundary_Condition'] == 'Ground':
            return 'Ground Floor'
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 'Exterior Floor'
        else:
            return 'Other'
        
    # Roofs & Ceilings
    if row['Surface_Type'] == 'Roof':
        return 'Roof'
    if row['Surface_Type'] == 'Ceiling':
        return 'Interior Floor'
    # Walls
    if row['Surface_Type'] == 'Wall':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 'Partition'
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 'Facade'
    return 'Other'

In [42]:
def type_surface(row):
    """
    This function adds the umi-Type column
    """
    # Floors
    if row['Surface_Type'] == 'Floor':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 3
        if row['Outside_Boundary_Condition'] == 'Ground':
            return 2
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 4
        else:
            return np.NaN
        
    # Roofs & Ceilings
    if row['Surface_Type'] == 'Roof':
        return 1
    if row['Surface_Type'] == 'Ceiling':
        return 3
    # Walls
    if row['Surface_Type'] == 'Wall':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 5
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 0
    return np.NaN

In [43]:
OpaqueConstructions['Category'] = OpaqueConstructions.apply(lambda x: label_surface(x), axis=1)
OpaqueConstructions['Type'] = OpaqueConstructions.apply(lambda x: type_surface(x), axis=1)

In [44]:
# Lets group by `Construction_Name` to get a list of unique `Surface_Type` and `Outside_Boundary_Condition`.

OpaqueConstructions = OpaqueConstructions.groupby('Construction_Name').first().reset_index()

In [45]:
def layer_composition(row, df):
    # Assumes 10 max layers
    layers = []
    
    # Let's start with the `Outside_Layer`
    ref, thickness = get_row_prop(row, df, 'Outside_Layer', 'Thickness')
    if thickness:
        layers.append({'Material':{'$ref':ref,'thickness':thickness}})
    else:
        thickness = 0.001 # Very small tickness
        layers.append({'Material':{'$ref':ref,'thickness':thickness}})
    # Then we iterate over the other layers. The number of layers is unknow. Limited to 10 for now
    for i in range(1,10):
        try:
            layer_name = 'Layer_{}'.format(i)
            ref, thickness = get_row_prop(row, df, layer_name, 'Thickness')
            if thickness:
                layers.append({'Material':{'$ref':ref,'thickness':thickness}})
            else:
                thickness = 0.001 # Very small tickness
                layers.append({'Material':{'$ref':ref,'thickness':thickness}})
        except:
            pass #
    return layers

In [46]:
def get_row_prop(row, df, column_name, prop):
    layer = df.loc[df['Name'] == row[column_name]]
    ref = layer.index[0]
    prop = layer[prop].values[0] # Very small tickness
    return ref, prop

In [47]:
OpaqueConstructions

,Construction_Name,Layer_2,Layer_3,Layer_4,Name_x,Name_y,Number_of_Vertices,Outside_Boundary_Condition,Outside_Boundary_Condition_Object,Outside_Layer,...,Vertex_9_Xcoordinate,Vertex_9_Ycoordinate,Vertex_9_Zcoordinate,View_Factor_to_Ground,Wind_Exposure,Zone_Name,key_x,key_y,Category,Type
0,Customized opaque construction {1292baf7-9fd7-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,Dining_Wall_East,Customized opaque construction {1292baf7-9fd7-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,,WindExposed,Sp-Dining Sys-3 Flr-1 Sch-B HPlcmt-south ZN,BuildingSurface:Detailed,Construction,Facade,0.0
1,Customized opaque construction {3a0bc7d3-1520-...,Typical Insulation R-19.63 1,NaN,NaN,Bath_ZN_1_FLR_1_Ceiling,Customized opaque construction {3a0bc7d3-1520-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,,WindExposed,Sp-Bath_ZN_1_FLR_1 Sys-3 Flr-1 Sch-D HPlcmt-we...,BuildingSurface:Detailed,Construction,Roof,1.0
2,Customized opaque construction {47144b0f-1dcc-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,Bath_ZN_1_FLR_1_Wall_5,Customized opaque construction {47144b0f-1dcc-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,,WindExposed,Sp-Bath_ZN_1_FLR_1 Sys-3 Flr-1 Sch-D HPlcmt-we...,BuildingSurface:Detailed,Construction,Facade,0.0
3,Customized opaque construction {4b8be161-d6b6-...,Typical Carpet Pad 2,NaN,NaN,g Floor C,Customized opaque construction {4b8be161-d6b6-...,,Ground,,6 in. Normalweight Concrete Floor 1,...,NaN,NaN,NaN,,NoWind,Sp-G Corridor Sys-1 Flr-5 Sch-G HPlcmt-east ZN,BuildingSurface:Detailed,Construction,Ground Floor,2.0
4,Customized opaque construction {70a08f41-941f-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,F2 EWall C,Customized opaque construction {70a08f41-941f-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,,WindExposed,Sp-F2 Corridor Sys-1 Flr-7 Sch-G HPlcmt-east ZN,BuildingSurface:Detailed,Construction,Facade,0.0
5,Customized opaque construction {8d7418df-8433-...,Typical Insulation R-19.63 1,NaN,NaN,t Roof C,Customized opaque construction {8d7418df-8433-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,,WindExposed,Sp-T Corridor Sys-1 Flr-4 Sch-G HPlcmt-east ZN,BuildingSurface:Detailed,Construction,Roof,1.0
6,Customized opaque construction {a4a6387c-c76b-...,8IN CONCRETE HW RefBldg 1,Typical Insulation R-10.11 1,1/2IN Gypsum 1,Corridor_Flr_1-1,Customized opaque construction {a4a6387c-c76b-...,,Outdoors,,1IN Stucco 1,...,NaN,NaN,NaN,,WindExposed,Sp-Corridor_Flr_1 Sys-6 Flr-1 Sch-B HPlcmt-nor...,BuildingSurface:Detailed,Construction,Facade,0.0
7,Customized opaque construction {d71d378f-b843-...,Typical Carpet Pad 2,NaN,NaN,Dining_Floor,Customized opaque construction {d71d378f-b843-...,,Ground,,6 in. Normalweight Concrete Floor 1,...,NaN,NaN,NaN,,NoWind,Sp-Dining Sys-3 Flr-1 Sch-B HPlcmt-south ZN,BuildingSurface:Detailed,Construction,Ground Floor,2.0
8,Customized opaque construction {dae5c91d-b614-...,Typical Insulation R-19.63 1,NaN,NaN,east-roof,Customized opaque construction {dae5c91d-b614-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,,WindExposed,Sp-attic Sys-0 Flr-2 Sch-- undefined - HPlcmt-...,BuildingSurface:Detailed,Construction,Roof,1.0
9,Customized opaque construction {dd65f4c9-6f6c-...,Typical Insulation R-19.63 1,NaN,NaN,Corridor_Flr_5-13,Customized opaque construction {dd65f4c9-6f6c-...,,Outdoors,,Metal Roof Surface 1,...,NaN,NaN,NaN,,WindExposed,Sp-Corridor_Flr_5 Sys-3 Flr-6 Sch-B HPlcmt-nor...,BuildingSurface:Detailed,Construction,Roof,1.0


In [48]:
OpaqueConstructions['Layers'] = OpaqueConstructions.apply(lambda x: layer_composition(x, OpaqueMaterials), axis=1)

In [ ]:
columns = ['Construction_Name', 'Category', 'Layers', 'Type']

In [ ]:
OpaqueConstructions = OpaqueConstructions[columns]

In [ ]:
OpaqueConstructions = OpaqueConstructions.rename(columns={'Construction_Name':'Name'})

In [ ]:
OpaqueConstructions['AssemblyCarbon'] = 0
OpaqueConstructions['AssemblyCost'] = 0
OpaqueConstructions['AssemblyEnergy'] = 0
OpaqueConstructions['Comments'] = 'default'
OpaqueConstructions['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]
OpaqueConstructions['DisassemblyCarbon'] = 0
OpaqueConstructions['DisassemblyEnergy'] = 0

In [ ]:
OpaqueConstructions.index = newrange(OpaqueMaterials,OpaqueConstructions)
OpaqueConstructions.index.name = '$id'

# WindowConstructions

In [ ]:
WindowConstructions_s = []
for idf in idfs: 
    WindowConstructions = idf.idfobjects['FENESTRATIONSURFACE:DETAILED']
    WindowConstructions = [get_values(frame) for frame in WindowConstructions]
    WindowConstructions = pd.concat(WindowConstructions, ignore_index=True, sort=True)
    WindowConstructions = WindowConstructions.merge(Construction, left_on='Construction_Name', right_on='Name')
    WindowConstructions_s.append(WindowConstructions)
# Groupby `Construction_Name` to keep only unique Window Constructions
WindowConstructions = pd.concat(WindowConstructions_s, keys=keys, names=['Archetype','$id'], sort=True)
WindowConstructions = WindowConstructions.groupby(['Archetype','Construction_Name']).first()
WindowConstructions.index = newrange(OpaqueConstructions,WindowConstructions)
WindowConstructions.index.name = '$id'

In [ ]:
idfs[0].idfobjects['FENESTRATIONSURFACE:DETAILED']

# DaySchedules

In [ ]:
from datetime import datetime, timedelta

In [ ]:
DaySchedules = []
for idf in idfs:
    DaySchedule = idf.idfobjects['SCHEDULE:DAY:INTERVAL']
    DaySchedule = [get_values(frame) for frame in DaySchedule]
    DaySchedule = pd.concat(DaySchedule, ignore_index=True, sort=True)
    DaySchedules.append(DaySchedule)
DaySchedule = pd.concat(DaySchedules, keys=keys, names=['Archetype','$id'], sort=True)
DaySchedule = DaySchedule.groupby('Name').first()
DaySchedule.reset_index(inplace=True)
DaySchedule.index = newrange(WindowConstructions, DaySchedule)
DaySchedule.index.name = '$id'

In [ ]:
def my_to_datetime(date_str):
    if date_str[0:2] != '24':
        return datetime.strptime(date_str, '%H:%M') - timedelta(hours=1)
    return datetime.strptime('23:00', '%H:%M')

In [ ]:
def time2time(row):
    time_seg = []
    for i in range(1,25):
        time = row['Time_{}'.format(i)] # Time_i
        value = row['Value_Until_Time_{}'.format(i)] # Value_Until_Time_i
        if str(time) != 'nan' and str(value) != 'nan':
#             print(time)
            time = my_to_datetime(time).hour
#             print(time)
            times = np.ones(time+1) * float(value)
            time_seg.append(times)
    arrays = time_seg
    array = time_seg[0]
    length = len(arrays[0])
    for i, a in enumerate(arrays):
        if i != 0:
            array = np.append(array, a[length-1:-1])
            length = len(a)
    return array

In [ ]:
DaySchedule['Values'] = DaySchedule.apply(lambda x: time2time(x), axis=1)

In [ ]:
DaySchedule = DaySchedule.loc[:,['Name', 'Values']]
DaySchedule.loc[:,'Category'] = 'Day'
DaySchedule.loc[:,'Comments'] = 'Comments'
DaySchedule.loc[:,'DataSource'] = 'default'
DaySchedule.loc[:,'Type'] = 'Fraction'
DaySchedule

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Plotting the schedules
for aid, grp in DaySchedule.groupby(['Name']):
    plt.plot(grp['Values'].values[0]) 

# WeekSchedule

In [ ]:
WeekSchedules = []
for idf in idfs:
    WeekSchedule = idf.idfobjects['SCHEDULE:WEEK:DAILY']
    WeekSchedule = [get_values(frame) for frame in WeekSchedule]
    WeekSchedule = pd.concat(WeekSchedule, ignore_index=True, sort=True)
    WeekSchedules.append(WeekSchedule)
WeekSchedule = pd.concat(WeekSchedules, keys=keys, names=['Archetype','$id'], sort=True)
WeekSchedule = WeekSchedule.groupby('Name').first()
WeekSchedule.reset_index(inplace=True)

In [ ]:
WeekSchedule

In [ ]:
def schedule_composition(row, df):
    # Assumes 7 days
    day_schedules = []
    days = ['Monday_ScheduleDay_Name',
           'Tuesday_ScheduleDay_Name',
           'Wednesday_ScheduleDay_Name',
           'Thursday_ScheduleDay_Name',
           'Friday_ScheduleDay_Name',
           'Saturday_ScheduleDay_Name',
           'Sunday_ScheduleDay_Name'] # With weekends last (as defined in umi-template)
    # Let's start with the `Outside_Layer`
    for day in days:
        try:
            ref, day_schedule = get_row_prop(row, df, day, 'Values')
            day_schedules.append({'$ref':ref})
        except:
            pass
    return day_schedules

In [ ]:
WeekSchedule['Values'] = WeekSchedule.apply(lambda x: schedule_composition(x, DaySchedule), axis=1)

In [ ]:
WeekSchedule = WeekSchedule[['Name','Values']]
WeekSchedule['Category'] = 'Week'
WeekSchedule['Comments'] = 'default'
WeekSchedule['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]

In [ ]:
WeekSchedule.index = newrange(DaySchedule, WeekSchedule)
WeekSchedule.index.name = '$id'
WeekSchedule

# YearSchedules

In [ ]:
YearSchedules = []
for idf in idfs:
    YearSchedule = idf.idfobjects['SCHEDULE:YEAR']
    YearSchedule = [get_values(frame) for frame in YearSchedule]
    YearSchedule = pd.concat(YearSchedule, ignore_index=True, sort=True)
    YearSchedules.append(YearSchedule)
YearSchedule = pd.concat(YearSchedules, keys=keys, names=['Archetype','$id'], sort=True)
YearSchedule = YearSchedule.groupby('Name').first()
YearSchedule.reset_index(inplace=True)

In [ ]:
def year_parts(row, df):
    ref, prop = get_row_prop(row, df, 'ScheduleWeek_Name_1', 'Name')
    fromday = row['Start_Day_1']
    frommonth = row['Start_Month_1']
    today = row['End_Day_1']
    tomonth = row['End_Month_1']
    parts = {'FromDay': fromday,
             'FromMonth': frommonth,
             'Schedule': {'$ref': ref},
             'ToDay': today,
             'ToMonth': tomonth}
    return parts

In [ ]:
YearSchedule['Parts'] = YearSchedule.apply(lambda x: year_parts(x, WeekSchedule), axis=1)

In [ ]:
YearSchedule

In [ ]:
YearSchedule = YearSchedule[['Name', 'Schedule_Type_Limits_Name','Parts']]

YearSchedule['Category'] = 'Year'
YearSchedule['Comments'] = 'default'
YearSchedule['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]

YearSchedule.index = newrange(WeekSchedule, YearSchedule)
YearSchedule.index.name = '$id'
YearSchedule

# Start of json READ

At this point, the needed information is easier to acces in the `qacq` files (json format)

In [ ]:
import json

In [ ]:
files = glob.glob("data/necb/NECB_2011_Montreal/*.json")

In [ ]:
# Reading the json as a dict
qaqcs = []
for file in files:
    with open(file) as json_data:
        qaqcs.append(json.load(json_data))

all_qaqcs = {}
for qaqc in qaqcs:
    qaqc_dfs = {}
    name = qaqc['building']['name']
    for key in qaqc.keys():
        if isinstance(qaqc[key], dict):
            try:
                qaqc_dfs[key] = pd.DataFrame.from_dict(qaqc[key])
            except Exception as inst:
                qaqc_dfs[key] = pd.DataFrame.from_dict([qaqc[key]])
        elif isinstance(qaqc[key], list):
            qaqc_dfs[key] = pd.DataFrame.from_dict(qaqc[key])
        else:
            qaqc_dfs[key] = qaqc[key]
    all_qaqcs[name] = qaqc_dfs

# Zones

Zones can be identified by the term `horzontal_placement` (norht, south, east, west, core) in the NECB building definition [https://github.com/NREL/OpenStudio-Prototype-Buildings/blob/master/lib/btap/measures/btap_equest_converter/compliance.rb]

In [ ]:
def iscore(row):
    """
    Helps to group by core and perimeter zones
    """
    if 'core' in row['thermal_zone'].lower(): # We look for the string `core` in the Zone_Name
        return 'Core'
    else:
        return 'Perimeter'

In [ ]:
Zones = []
for qaqc in all_qaqcs.values():
    Zone = qaqc['spaces']
    Zones.append(Zone)
Zones = pd.concat(Zones, keys=all_qaqcs.keys(), names=['Archetype','$id'], sort=True)
Zones['Space_Type'] = Zones.apply(lambda x: iscore(x), axis=1)

In [ ]:
def ach(row):
    """
    Calculates Air changes per hour (ACH)
    """
    q = row['infiltration_flow_m3_per_s']
    V = row['volume']
    if q > 0:
        return 3600 * q / V
    return np.NaN

In [ ]:
# Calculating the infiltration flow by use of the area-infiltration flow times the exterior wall area
Zones['infiltration_flow_m3_per_s'] = Zones['infiltration_flow_per_m2'] * Zones['exterior_wall_area']

In [ ]:
Zones['air_changes_per_hour'] = Zones.apply(lambda x: ach(x), axis=1)

In [ ]:
# Let's add the space area

spacetype_area_breakdown = []
for qaqc in all_qaqcs.values():
    area = qaqc['spacetype_area_breakdown'].T.rename(columns={0:'Space_Area'})
    spacetype_area_breakdown.append(area)
spacetype_area_breakdown = pd.concat(spacetype_area_breakdown, axis=0, keys=all_qaqcs.keys(), names=['Archetype','space_type_name'], sort=True)

In [ ]:
Zones['space_type_name'] = Zones['space_type_name'].str.replace(' ','_').str.lower()

In [ ]:
Zones = Zones.reset_index().set_index(['Archetype','space_type_name']).merge(spacetype_area_breakdown, left_index=True, right_index=True, how='left')
Zones = Zones.reset_index().set_index(['Archetype','$id'])

In [ ]:
Zones

In [ ]:
# Let's add a the `waterUseEquipment` as columns instead of the object they are in the DataFrame.
# To do this, we merge a new dataframe that is created with the apply(pd.Series) function. This transforms
# the dict stucture of each rows into a the differetn columns. The apply the Pd.Series fucntion twice because
# the dicts are inside a list.

Zones = Zones.join(Zones['waterUseEquipment'].apply(pd.Series)[0].apply(pd.Series), sort=True)

In [ ]:
def weighted_mean(series):
    """
    Evaluates a weighteed average while ignoring NaNs
    """
    index = ~np.isnan(series)
    if np.any(index):
        weights=Zones.loc[series.index, 'volume'] * Zones.loc[series.index, 'multiplier']
        weights=weights[index]
        a = series[index]
        return np.average(a, weights=weights)
    return 0

In [ ]:
# Define a lambda function to compute the weighted mean:
# wm = lambda x: np.average(~np.isnan(x), weights=(Zones.loc[x.index, 'volume'] * Zones.loc[x.index, 'multiplier'])[~np.isnan(x)])
wu = lambda x: x.apply(pd.Series)
# Define a dictionary with the functions to apply for a given column:
f = {'air_changes_per_hour': {'weighted_mean' : weighted_mean},
     'occ_per_m2' : {'weighted_mean' : weighted_mean},
     'breathing_zone_outdoor_airflow_vbz' : {'weighted_mean' : weighted_mean},
     'electric_w_per_m2' : {'weighted_mean' : weighted_mean},
     'lighting_w_per_m2' : {'weighted_mean' : weighted_mean},
     'peak_flow_rate_per_area' : {'weighted_mean' : weighted_mean},
     'Space_Area' : {'sum' : 'sum'}
    }

# Groupby and aggregate with your dictionary:
ZonesByType = Zones.groupby(['Archetype','Space_Type'], sort=True).agg(f).reset_index()
ZonesByType['FlowRatePerFloorArea'] = ZonesByType['peak_flow_rate_per_area'] * 36000 # m3/s/m2 to m3/h/m2
ZonesByType

# DomesticHotWaterSettings

In [ ]:
# Category
# DataSource
# FlowRatePerFloorArea
# IsOn
# Name
# WaterSchedule.$ref - Source: Probably in idf file
# WaterSupplyTemperature
# WaterTemperatureInlet - Source: Probably in idf file
DomesticHotWaterSettingsColumns = ['Category', 'DataSource', 'FlowRatePerFloorArea', 'IsOn', 'Name',
       'WaterSchedule.$ref', 'WaterSupplyTemperature',
       'WaterTemperatureInlet']
# Creating a DataFrame

possibleSchedules = DaySchedule[DaySchedule.Name.str.contains('Service Water Loop Temp', case=False)]

# WaterSchedule.$ref
WaterSchedule_ref = YearSchedule[YearSchedule.Name.str.contains('Service Water Loop Temp', case=False)].index

# WaterSupplyTemperature
waterschedule = DaySchedule.reset_index().set_index('Name').loc[possibleSchedules.Name.values,:]
WaterSupplyTemperature = waterschedule.Values.mean().mean() # the first mean() averages possible multiple schedules, the second mean(gets the value

# WaterTemperatureInlet - Source: Probably in idf file
WaterTemperatureInlet = 8.0 # ! This needs to be investigated

DomesticHotWaterSettings = ZonesByType.loc[:,['Archetype', 'Space_Type','FlowRatePerFloorArea']]
DomesticHotWaterSettings.loc[:,'Category'] = DomesticHotWaterSettings['Space_Type']
DomesticHotWaterSettings.loc[:,'IsOn'] = DomesticHotWaterSettings.apply(lambda x: x['FlowRatePerFloorArea'] > 0, axis=1)
DomesticHotWaterSettings.loc[:,'Name'] = DomesticHotWaterSettings['Archetype'] + '_' + DomesticHotWaterSettings['Space_Type']
DomesticHotWaterSettings.loc[:,'DataSource'] = DomesticHotWaterSettings['Archetype']
DomesticHotWaterSettings.loc[:,'WaterSchedule.$ref'] = WaterSchedule_ref.values[0]
DomesticHotWaterSettings.loc[:,'WaterSupplyTemperature'] = WaterSupplyTemperature
DomesticHotWaterSettings.loc[:,'WaterTemperatureInlet'] = WaterTemperatureInlet
DomesticHotWaterSettings.index = newrange(YearSchedule, DomesticHotWaterSettings)
DomesticHotWaterSettings.index.name = '$id'
DomesticHotWaterSettings = DomesticHotWaterSettings[DomesticHotWaterSettingsColumns]

In [ ]:
DomesticHotWaterSettings

# VentilationSettings

In [ ]:
# 'Afn',
# 'Category',
# 'DataSource',
# 'Infiltration',
# 'IsBuoyancyOn',
# 'IsInfiltrationOn',
# 'IsNatVentOn',
# 'IsScheduledVentilationOn',
# 'IsWindOn',
# 'Name',
# 'NatVentMaxOutdoorAirTemp',
# 'NatVentMaxRelHumidity',
# 'NatVentMinOutdoorAirTemp',
# 'NatVentSchedule.$ref',
# 'NatVentZoneTempSetpoint',
# 'ScheduledVentilationAch',
# 'ScheduledVentilationSchedule.$ref',
# 'ScheduledVentilationSetpoint'

VentilationSettingsSettingsColumns = ['Afn', 'Category', 'DataSource', 'Infiltration', 'IsBuoyancyOn',
       'IsInfiltrationOn', 'IsNatVentOn', 'IsScheduledVentilationOn',
       'IsWindOn', 'Name', 'NatVentMaxOutdoorAirTemp', 'NatVentMaxRelHumidity',
       'NatVentMinOutdoorAirTemp', 'NatVentSchedule.$ref',
       'NatVentZoneTempSetpoint', 'ScheduledVentilationAch',
       'ScheduledVentilationSchedule.$ref', 'ScheduledVentilationSetpoint']

In [ ]:
VentilationSettings = ZonesByType.loc[:,['Archetype', 'Space_Type']]

In [ ]:
VentilationSettings.loc[:,'Infiltration'] = ZonesByType.loc[:,'air_changes_per_hour'].values
VentilationSettings.loc[:,'IsBuoyancyOn'] = True
VentilationSettings.loc[:,'IsInfiltrationOn'] = VentilationSettings.apply(lambda x: x.loc['Infiltration'] > 0, axis=1)
VentilationSettings.loc[:,'IsNatVentOn'] = False
VentilationSettings.loc[:,'IsScheduledVentilationOn'] = False
VentilationSettings.loc[:,'IsWindOn'] = False
VentilationSettings.loc[:,'Name'] = VentilationSettings['Archetype'] + '_' + VentilationSettings['Space_Type']
VentilationSettings.loc[:,'NatVentMaxOutdoorAirTemp'] = 26.0
VentilationSettings.loc[:,'NatVentMaxRelHumidity'] = 80.0
VentilationSettings.loc[:,'NatVentMinOutdoorAirTemp'] = 18.0

In [ ]:
VentilationSettings

In [ ]:
YearSchedule